In [12]:
import typing
import itertools
from PIL import Image
import weave

## Show an image

In [13]:
weave.show(Image.linear_gradient("L"))

## Images in Lists

In [15]:
ims = [Image.linear_gradient("L").rotate(i * 4) for i in range(100)]
weave.show(ims)

## Images in Tables

In [16]:
ims = []
base_im = Image.linear_gradient('L')
for rotate in range(10):
    for shear in range(10):
        ims.append({
            'rotate': rotate,
            'shear': shear,
            'image': base_im
                .rotate(rotate * 4)
                .transform((256, 256), Image.AFFINE, (1, shear / 10, 0, 0, 1, 0), Image.BICUBIC)
        })
weave.show(ims)

## Facet is fun!

In [17]:
facet = weave.panels.Facet(ims)
facet.set_x(lambda row: row['rotate'])
facet.set_y(lambda row: row['shear'])
facet.set_select(lambda row: row[0]['image'])
weave.show(facet, height=600)

## Create some ops to make it dynamic and interactive

In [18]:
class ParamType(typing.TypedDict):
    rotate: int
    shear: int
        
@weave.op(render_info={'type': 'function'})
def make_params(n_rotate: int, n_shear: int) -> list[ParamType]:
    params = []
    for rotate, shear in itertools.product(range(n_rotate), range(n_shear)):
        params.append({
            'rotate': rotate,
            'shear': shear
        })
    return params

@weave.op(render_info={'type': 'function'})
def make_image(rotate: int, shear: int) -> Image.Image:
    return (Image.linear_gradient('L')
        .rotate(rotate * 4)
        .transform((256, 256), Image.AFFINE, (1, shear / 10, 0, 0, 1, 0), Image.BICUBIC))

facet = weave.panels.Facet(make_params(10, 5))
facet.set_x(lambda row: row['rotate'])
facet.set_y(lambda row: row['shear']) 
facet.set_select(lambda row: make_image(row[0]['rotate'], row[0]['shear']))
weave.show(facet, height=600)